Generated from: intent_training.ipynb
Converted at: 2026-01-23T05:56:13.369Z
Next step (optional): refactor into modules & generate tests with RunCell
Quick start: pip install runcell

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix, confusion_matrix
import re
from sentence_transformers import SentenceTransformer

In [2]:
file_path = "data/intent.xlsx"
data = pd.read_excel(file_path)

In [3]:
data = data [["text", "intent"]]
data.head()

,text,intent
0,halo mlibbot,salam
1,selamat pagi perpustakaan,salam
2,hai selamat siang,salam
3,makasih ya mlibbot,salam
4,terima kasih atas bantuannya,salam


In [5]:
print(data.columns)
data["intent"].value_counts()

Index(['text', 'intent'], dtype='object')


intent
salam                        68
tanya_fungsi_mlibbot         65
cari_buku_judul              65
cari_buku_penulis            65
cari_buku_topik              65
cari_buku_isbn_callnumber    65
cek_ketersediaan_buku        65
lokasi_buku_rak              65
jam_buka                     65
lokasi_perpustakaan          65
panduan_peminjaman           65
panduan_pengembalian         65
panduan_perpanjangan         65
info_denda                   65
tata_tertib                  65
layanan_ruang_diskusi        65
layanan_ejournal_ebook       65
layanan_turnitin             65
donasi_buku                  65
akses_repository             65
cari_rekomendasi             65
lainnya                      65
Name: count, dtype: int64

In [6]:
def preprocess(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    text = re.sub(r"[^0-9a-zA-ZÀ-ÿ\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [7]:
data["hasil"] = data["text"].apply(preprocess)
data[["text", "hasil"]].head(20)

,text,hasil
0,halo mlibbot,halo mlibbot
1,selamat pagi perpustakaan,selamat pagi perpustakaan
2,hai selamat siang,hai selamat siang
3,makasih ya mlibbot,makasih ya mlibbot
4,terima kasih atas bantuannya,terima kasih atas bantuannya
5,halo,halo
6,hai,hai
7,hey,hey
8,kamu itu bisa bantu apa aja sih,kamu itu bisa bantu apa aja sih
9,mlibbot fungsinya apa,mlibbot fungsinya apa


In [8]:
data.isnull().sum()

text      0
intent    0
hasil     0
dtype: int64

In [9]:
data = data.dropna()

In [10]:
data.isnull().sum()

text      0
intent    0
hasil     0
dtype: int64

In [11]:
data

,text,intent,hasil
0,halo mlibbot,salam,halo mlibbot
1,selamat pagi perpustakaan,salam,selamat pagi perpustakaan
2,hai selamat siang,salam,hai selamat siang
3,makasih ya mlibbot,salam,makasih ya mlibbot
4,terima kasih atas bantuannya,salam,terima kasih atas bantuannya
...,...,...,...
1428,"buat nemenin praktikum basis data, enak klo ad...",cari_rekomendasi,buat nemenin praktikum basis data enak klo ada...
1429,sy suka bku yg bahas teori trus lanjut studi k...,cari_rekomendasi,sy suka bku yg bahas teori trus lanjut studi k...
1430,"gw lg bosen baca modul doang, pengen ganti sua...",cari_rekomendasi,gw lg bosen baca modul doang pengen ganti suas...
1431,"sy ngerasa perlu satu bacaan utama soal UI UX,...",cari_rekomendasi,sy ngerasa perlu satu bacaan utama soal ui ux ...


In [12]:
data = data[["hasil", "intent"]]

In [13]:
data

,hasil,intent
0,halo mlibbot,salam
1,selamat pagi perpustakaan,salam
2,hai selamat siang,salam
3,makasih ya mlibbot,salam
4,terima kasih atas bantuannya,salam
...,...,...
1428,buat nemenin praktikum basis data enak klo ada...,cari_rekomendasi
1429,sy suka bku yg bahas teori trus lanjut studi k...,cari_rekomendasi
1430,gw lg bosen baca modul doang pengen ganti suas...,cari_rekomendasi
1431,sy ngerasa perlu satu bacaan utama soal ui ux ...,cari_rekomendasi


In [14]:
X = data["hasil"].astype(str).tolist()
y = data["intent"].astype(str).tolist()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
len(X_train), len(X_test)

(1146, 287)

In [17]:
INDOBERT_MODEL_NAME = "LazarusNLP/all-indobert-base-v4"
class IndoBertEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name=INDOBERT_MODEL_NAME, batch_size=32):
        self.model_name = model_name
        self.batch_size = batch_size
        self.model = None

    def fit(self, X, y=None):
        if self.model is None:
            self.model = SentenceTransformer(self.model_name)
        return self

    def transform(self, X):
        embeddings = self.model.encode(
            X,
            batch_size=self.batch_size,
            convert_to_numpy=True,
            show_progress_bar=False,
            normalize_embeddings=True,  
        ).astype(np.float32)
        return embeddings

In [18]:
pipe_logreg = Pipeline([
    ("tfidf", TfidfVectorizer(
        preprocessor=None,    
        lowercase=False        
    )),
    ("clf", LogisticRegression(
        max_iter=500,
        n_jobs=-1
    ))
])

In [19]:
param_grid_logreg = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [1, 2],
    "clf__C": [0.1, 1.0, 5.0]
}

In [20]:
grid_logreg = GridSearchCV(
    pipe_logreg,
    param_grid_logreg,
    cv=5,
    n_jobs=-1,
    verbose=2
)

In [21]:
grid_logreg.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\deivi\anaconda3\envs\mlibbot\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step... n_jobs=-1))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'clf__C': [0.1, 1.0, ...], 'tfidf__min_df': [1, 2], 'tfidf__ngram_range': [(1, ...), (1, ...)]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >

In [22]:
print("Best params (LogReg):", grid_logreg.best_params_)
print("Best CV score (LogReg):", grid_logreg.best_score_)

Best params (LogReg): {'clf__C': 5.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Best CV score (LogReg): 0.7949610784127588


In [23]:
best_logreg = grid_logreg.best_estimator_

In [24]:
y_pred_logreg = best_logreg.predict(X_test)
acc_logreg = accuracy_score(y_test, y_pred_logreg)
print(f"Test Accuracy (LogReg TF-IDF): {acc_logreg:.3f}\n")

Test Accuracy (LogReg TF-IDF): 0.826



In [25]:
print("Classification Report (LogReg TF-IDF):")
print(classification_report(y_test, y_pred_logreg))

Classification Report (LogReg TF-IDF):
                           precision    recall  f1-score   support

         akses_repository       0.79      0.85      0.81        13
cari_buku_isbn_callnumber       0.91      0.77      0.83        13
          cari_buku_judul       0.75      0.92      0.83        13
        cari_buku_penulis       0.64      0.54      0.58        13
          cari_buku_topik       0.91      0.77      0.83        13
         cari_rekomendasi       0.87      1.00      0.93        13
    cek_ketersediaan_buku       0.90      0.69      0.78        13
              donasi_buku       0.64      0.69      0.67        13
               info_denda       0.89      0.62      0.73        13
                 jam_buka       0.92      0.85      0.88        13
                  lainnya       0.72      1.00      0.84        13
   layanan_ejournal_ebook       0.80      0.92      0.86        13
    layanan_ruang_diskusi       0.93      1.00      0.96        13
         layanan_turni

In [26]:
# Analisis TP, FP, FN, TN per Intent (LogReg TF-IDF)
def analyze_intent_difficulty(y_true, y_pred, model):
    labels = model.classes_
    mcm = multilabel_confusion_matrix(y_true, y_pred, labels=labels)
    
    results = []
    for i, intent in enumerate(labels):
        tn, fp, fn, tp = mcm[i].ravel()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        results.append({
            "Intent": intent,
            "TP": tp, "FP": fp, "FN": fn, "TN": tn,
            "Precision": round(precision, 4),
            "Recall": round(recall, 4),
            "F1-Score": round(f1, 4)
        })
    return pd.DataFrame(results).sort_values("F1-Score", ascending=True)

In [27]:
print("\nAnalisis Intent Paling Sulit (LogReg TF-IDF):")
df_logreg = analyze_intent_difficulty(y_test, y_pred_logreg, best_logreg)
print(df_logreg[["Intent", "F1-Score", "Precision", "Recall", "TP", "FP", "FN"]].head(5).to_string(index=False))


Analisis Intent Paling Sulit (LogReg TF-IDF):
              Intent  F1-Score  Precision  Recall  TP  FP  FN
   cari_buku_penulis    0.5833     0.6364  0.5385   7   4   6
         donasi_buku    0.6667     0.6429  0.6923   9   5   4
     lokasi_buku_rak    0.6667     0.5500  0.8462  11   9   2
          info_denda    0.7273     0.8889  0.6154   8   1   5
panduan_pengembalian    0.7273     0.8889  0.6154   8   1   5


In [28]:
pipe_logreg_indobert = Pipeline([
    ("indobert", IndoBertEncoder(
        model_name=INDOBERT_MODEL_NAME,
        batch_size=32
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    ))
])

In [29]:
param_grid_logreg_indobert = {
    "clf__C": [0.1, 1.0, 5.0],
    "clf__class_weight": [None, "balanced"],
}

In [30]:
grid_logreg_indobert = GridSearchCV(
    pipe_logreg_indobert,
    param_grid_logreg_indobert,
    cv=5,
    n_jobs=-1,
    verbose=2
)

In [31]:
grid_logreg_indobert.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\deivi\anaconda3\envs\mlibbot\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step... n_jobs=-1))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'clf__C': [0.1, 1.0, ...], 'clf__class_weight': [None, 'balanced']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displa

In [32]:
print("Best params (LogReg):", grid_logreg_indobert.best_params_)
print("Best CV score (LogReg):", grid_logreg_indobert.best_score_)

Best params (LogReg): {'clf__C': 5.0, 'clf__class_weight': 'balanced'}
Best CV score (LogReg): 0.7617846971710651


In [33]:
best_logreg_indobert = grid_logreg_indobert.best_estimator_

In [34]:
y_pred_logreg_indobert = best_logreg_indobert.predict(X_test)
acc_logreg_indobert = accuracy_score(y_test, y_pred_logreg_indobert)
print(f"Test Accuracy (LogReg IndoBERT): {acc_logreg_indobert:.3f}\n")

Test Accuracy (LogReg IndoBERT): 0.767



In [35]:
print("Classification Report (LogReg IndoBERT):")
print(classification_report(y_test, y_pred_logreg_indobert))

Classification Report (LogReg IndoBERT):
                           precision    recall  f1-score   support

         akses_repository       0.77      0.77      0.77        13
cari_buku_isbn_callnumber       0.92      0.85      0.88        13
          cari_buku_judul       0.56      0.77      0.65        13
        cari_buku_penulis       0.67      0.77      0.71        13
          cari_buku_topik       0.75      0.46      0.57        13
         cari_rekomendasi       0.77      0.77      0.77        13
    cek_ketersediaan_buku       0.70      0.54      0.61        13
              donasi_buku       0.80      0.92      0.86        13
               info_denda       1.00      0.85      0.92        13
                 jam_buka       0.77      0.77      0.77        13
                  lainnya       0.62      0.77      0.69        13
   layanan_ejournal_ebook       0.62      0.62      0.62        13
    layanan_ruang_diskusi       0.86      0.92      0.89        13
         layanan_tur

In [36]:
print("\nAnalisis Intent Paling Sulit (LogReg IndoBERT):")
df_indobert = analyze_intent_difficulty(y_test, y_pred_logreg_indobert, best_logreg_indobert)
print(df_indobert[["Intent", "F1-Score", "Precision", "Recall", "TP", "FP", "FN"]].head(5).to_string(index=False))


Analisis Intent Paling Sulit (LogReg IndoBERT):
                Intent  F1-Score  Precision  Recall  TP  FP  FN
       cari_buku_topik    0.5714     0.7500  0.4615   6   2   7
 cek_ketersediaan_buku    0.6087     0.7000  0.5385   7   3   6
layanan_ejournal_ebook    0.6154     0.6154  0.6154   8   5   5
    panduan_peminjaman    0.6154     0.6154  0.6154   8   5   5
       cari_buku_judul    0.6452     0.5556  0.7692  10   8   3


In [37]:
pipe_nb = Pipeline([
    ("tfidf", TfidfVectorizer(
        preprocessor=None,
        lowercase=False
    )),
    ("clf", MultinomialNB())
])

In [38]:
param_grid_nb = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [1, 2],
    "clf__alpha": [0.1, 0.5, 1.0]
}

In [39]:
grid_nb = GridSearchCV(
    pipe_nb,
    param_grid_nb,
    cv=5,
    n_jobs=-1,
    verbose=2
)

In [40]:
grid_nb.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...inomialNB())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'clf__alpha': [0.1, 0.5, ...], 'tfidf__min_df': [1, 2], 'tfidf__ngram_range': [(1, ...), (1, ...)]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed

In [41]:
print("Best params (NB):", grid_nb.best_params_)
print("Best CV score (NB):", grid_nb.best_score_)

Best params (NB): {'clf__alpha': 0.1, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}
Best CV score (NB): 0.7731573951015759


In [42]:
best_nb = grid_nb.best_estimator_

In [43]:
y_pred_nb = best_nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
print(f"Test Accuracy (Naive Bayes): {acc_nb:.3f}\n")

Test Accuracy (Naive Bayes): 0.787



In [44]:
print("Classification Report (Naive Bayes):")
print(classification_report(y_test, y_pred_nb))

Classification Report (Naive Bayes):
                           precision    recall  f1-score   support

         akses_repository       0.73      0.85      0.79        13
cari_buku_isbn_callnumber       0.86      0.92      0.89        13
          cari_buku_judul       0.86      0.92      0.89        13
        cari_buku_penulis       0.73      0.62      0.67        13
          cari_buku_topik       0.71      0.77      0.74        13
         cari_rekomendasi       0.85      0.85      0.85        13
    cek_ketersediaan_buku       0.90      0.69      0.78        13
              donasi_buku       0.57      0.62      0.59        13
               info_denda       0.78      0.54      0.64        13
                 jam_buka       0.92      0.92      0.92        13
                  lainnya       0.75      0.92      0.83        13
   layanan_ejournal_ebook       0.77      0.77      0.77        13
    layanan_ruang_diskusi       0.92      0.92      0.92        13
         layanan_turniti

In [45]:
print("\nAnalisis Intent Paling Sulit (Naive Bayes):")
df_nb = analyze_intent_difficulty(y_test, y_pred_nb, best_nb)
print(df_nb[["Intent", "F1-Score", "Precision", "Recall", "TP", "FP", "FN"]].head(5).to_string(index=False))


Analisis Intent Paling Sulit (Naive Bayes):
              Intent  F1-Score  Precision  Recall  TP  FP  FN
panduan_pengembalian    0.5600     0.5833  0.5385   7   5   6
         donasi_buku    0.5926     0.5714  0.6154   8   6   5
          info_denda    0.6364     0.7778  0.5385   7   2   6
   cari_buku_penulis    0.6667     0.7273  0.6154   8   3   5
         tata_tertib    0.6957     0.8000  0.6154   8   2   5


Pada percobaan ini, model Naive Bayes tetap menggunakan fitur TF-IDF dan tidak digabung dengan embedding IndoBERT. Alasannya karena secara prinsip, Multinomial Naive Bayes dirancang untuk bekerja dengan fitur berupa frekuensi kata atau bobot yang mirip frekuensi (seperti count dan TF-IDF) yang bernilai non-negatif.

Sementara itu, embedding IndoBERT berbentuk vektor dens dengan nilai kontinu yang bisa positif maupun negatif, dan tidak lagi merepresentasikan "jumlah kemunculan kata", tetapi makna kalimat di ruang vektor. Tipe fitur seperti ini tidak sesuai dengan asumsi probabilistik Multinomial Naive Bayes, sehingga performanya justru bisa tidak stabil atau menurun.

In [46]:
print(f"LogReg (TF-IDF) Test Accuracy      : {acc_logreg:.3f}")
print(f"Naive Bayes (TF-IDF) Test Accuracy : {acc_nb:.3f}")
print(f"LogReg (IndoBERT) Test Accuracy    : {acc_logreg_indobert:.3f}")

LogReg (TF-IDF) Test Accuracy      : 0.826
Naive Bayes (TF-IDF) Test Accuracy : 0.787
LogReg (IndoBERT) Test Accuracy    : 0.767


In [47]:
candidates = {
    "logreg_tfidf":      (acc_logreg, best_logreg),
    "naive_bayes_tfidf": (acc_nb, best_nb),
    "logreg_indobert":   (acc_logreg_indobert, best_logreg_indobert),
}

In [48]:
best_model_name, (best_acc, final_model) = max(
    candidates.items(),
    key=lambda item: item[1][0]  
)

In [49]:
print(f"Chosen model: {best_model_name} (accuracy = {best_acc:.3f})")

Chosen model: logreg_tfidf (accuracy = 0.826)


In [50]:
# Simpan semua model
print("\nMenyimpan semua model...")


Menyimpan semua model...


In [51]:
logreg_tfidf_path = "model/intent_model_logreg_tfidf.pkl"
joblib.dump(best_logreg, logreg_tfidf_path)
print(f"Saved LogReg TF-IDF to: {logreg_tfidf_path}")

Saved LogReg TF-IDF to: model/intent_model_logreg_tfidf.pkl


In [52]:
nb_tfidf_path = "model/intent_model_naive_bayes_tfidf.pkl"
joblib.dump(best_nb, nb_tfidf_path)
print(f"Saved Naive Bayes TF-IDF to: {nb_tfidf_path}")

Saved Naive Bayes TF-IDF to: model/intent_model_naive_bayes_tfidf.pkl


In [53]:
indobert_path = "model/intent_model_logreg_indobert.pkl"
joblib.dump(best_logreg_indobert, indobert_path)
print(f"Saved LogReg IndoBERT to: {indobert_path}")

Saved LogReg IndoBERT to: model/intent_model_logreg_indobert.pkl


In [54]:
def predict_intent_sentence(s):
    s_clean = preprocess(s)
    return final_model.predict([s_clean])[0]

In [55]:
tests = [
    "jam buka perpustakaan hari sabtu",
    "perpus maranatha buka sampe jam berapa ya?",
    "besok minggu perpus buka gak?",
    "jam operasional perpustakaan pas libur nasional gimana?",
    "hari ini perpus udah buka belum?",

    "ada buku basis data fathansyah gak",
    "ada buku tentang machine learning terbaru gak?",
    "cek dong buku pemrograman python masih tersedia ga",
    "di perpus ada novel laskar pelangi gak sih?",
    "kalo mau cari skripsi tentang data mining ada ga?",

    "cara booking ruang diskusi gimana",
    "book ruang belajar kelompok bisa lewat mana?",
    "ruang diskusi bisa dipake berapa jam maksimal?",
    "bisa reservasi ruang belajar lewat online gak?",

    "kalau telat balikin buku dendanya berapa",
    "telat ngembaliin buku 2 hari berapa ya?",
    "Kalau saya telat mengembalikan, konsekuensinya apa?",
    "kalau hilangin buku perpus dendanya gimana ya?",
    "batas maksimal telat pengembalian sebelum kena blokir berapa hari?",

    "cara akses e journal dari luar kampus",
    "akses database journal lewat wifi kos bisa gak?",
    "punya akses ke ieee atau sciencedirect gak ya?",
    "login e-resources pake akun apa ya?",
    "kalo lupa password e journal harus gimana?",

    "perpus maranatha ada dmn sih",
    "alamat lengkap perpustakaan maranatha di mana ya?",
    "nomor telepon perpustakaan ada?",
    "perpus ada di gedung mana ya di kampus?",

    "cara pinjam buku di perpus gimana",
    "bisa perpanjang peminjaman buku lewat online gak?",
    "kalo mau pinjem buku harus bawa ktm gak?",
    "maksimal bisa pinjam berapa buku sekaligus?",
    "lama peminjaman buku berapa hari ya?",

    "halo mlibbot",
    "hi bot, bisa bantu cari buku?",
    "p",
    "halo, ini perpus maranatha ya?",
]

In [56]:
for t in tests:
    print(f"{t!r} -> {predict_intent_sentence(t)}")

'jam buka perpustakaan hari sabtu' -> jam_buka
'perpus maranatha buka sampe jam berapa ya?' -> jam_buka
'besok minggu perpus buka gak?' -> jam_buka
'jam operasional perpustakaan pas libur nasional gimana?' -> jam_buka
'hari ini perpus udah buka belum?' -> jam_buka
'ada buku basis data fathansyah gak' -> cari_buku_judul
'ada buku tentang machine learning terbaru gak?' -> cari_rekomendasi
'cek dong buku pemrograman python masih tersedia ga' -> cek_ketersediaan_buku
'di perpus ada novel laskar pelangi gak sih?' -> cari_buku_judul
'kalo mau cari skripsi tentang data mining ada ga?' -> cari_buku_topik
'cara booking ruang diskusi gimana' -> layanan_ruang_diskusi
'book ruang belajar kelompok bisa lewat mana?' -> layanan_ruang_diskusi
'ruang diskusi bisa dipake berapa jam maksimal?' -> layanan_ruang_diskusi
'bisa reservasi ruang belajar lewat online gak?' -> panduan_perpanjangan
'kalau telat balikin buku dendanya berapa' -> info_denda
'telat ngembaliin buku 2 hari berapa ya?' -> info_denda
'Ka

Test dengan IndoBERT model

In [57]:
def predict_intent_sentence(s):
    s_clean = preprocess(s)
    return best_logreg_indobert.predict([s_clean])[0]

In [58]:
tests = [
    "jam buka perpustakaan hari sabtu",
    "perpus maranatha buka sampe jam berapa ya?",
    "besok minggu perpus buka gak?",
    "jam operasional perpustakaan pas libur nasional gimana?",
    "hari ini perpus udah buka belum?",

    "ada buku basis data fathansyah gak",
    "ada buku tentang machine learning terbaru gak?",
    "cek dong buku pemrograman python masih tersedia ga",
    "di perpus ada novel laskar pelangi gak sih?",
    "kalo mau cari skripsi tentang data mining ada ga?",

    "cara booking ruang diskusi gimana",
    "book ruang belajar kelompok bisa lewat mana?",
    "ruang diskusi bisa dipake berapa jam maksimal?",
    "bisa reservasi ruang belajar lewat online gak?",

    "kalau telat balikin buku dendanya berapa",
    "telat ngembaliin buku 2 hari berapa ya?",
    "Kalau saya telat mengembalikan, konsekuensinya apa?",
    "kalau hilangin buku perpus dendanya gimana ya?",
    "batas maksimal telat pengembalian sebelum kena blokir berapa hari?",

    "cara akses e journal dari luar kampus",
    "akses database journal lewat wifi kos bisa gak?",
    "punya akses ke ieee atau sciencedirect gak ya?",
    "login e-resources pake akun apa ya?",
    "kalo lupa password e journal harus gimana?",

    "perpus maranatha ada dmn sih",
    "alamat lengkap perpustakaan maranatha di mana ya?",
    "nomor telepon perpustakaan ada?",
    "perpus ada di gedung mana ya di kampus?",

    "cara pinjam buku di perpus gimana",
    "bisa perpanjang peminjaman buku lewat online gak?",
    "kalo mau pinjem buku harus bawa ktm gak?",
    "maksimal bisa pinjam berapa buku sekaligus?",
    "lama peminjaman buku berapa hari ya?",

    "halo mlibbot",
    "hi bot, bisa bantu cari buku?",
    "p",
    "halo, ini perpus maranatha ya?",
]

In [59]:
for t in tests:
    print(f"{t!r} -> {predict_intent_sentence(t)}")

'jam buka perpustakaan hari sabtu' -> jam_buka
'perpus maranatha buka sampe jam berapa ya?' -> jam_buka
'besok minggu perpus buka gak?' -> salam
'jam operasional perpustakaan pas libur nasional gimana?' -> jam_buka
'hari ini perpus udah buka belum?' -> salam
'ada buku basis data fathansyah gak' -> cari_buku_penulis
'ada buku tentang machine learning terbaru gak?' -> cari_buku_judul
'cek dong buku pemrograman python masih tersedia ga' -> cari_buku_judul
'di perpus ada novel laskar pelangi gak sih?' -> cari_buku_penulis
'kalo mau cari skripsi tentang data mining ada ga?' -> cari_buku_judul
'cara booking ruang diskusi gimana' -> layanan_ruang_diskusi
'book ruang belajar kelompok bisa lewat mana?' -> layanan_ruang_diskusi
'ruang diskusi bisa dipake berapa jam maksimal?' -> layanan_ruang_diskusi
'bisa reservasi ruang belajar lewat online gak?' -> layanan_ejournal_ebook
'kalau telat balikin buku dendanya berapa' -> info_denda
'telat ngembaliin buku 2 hari berapa ya?' -> panduan_perpanjangan
